In [1]:
# pip install keras-self-attention
import os, time, gc, sys, io
import datetime, pickle, session_info
import warnings, random, math
#
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import mpl_toolkits.axisartist as AA
#
from scipy import stats
from pandas import Series
from argparse import ArgumentParser
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import GRU
from keras.layers import Bidirectional
from keras.layers import Flatten
from keras.layers import Activation
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.layers import Embedding
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from keras.models import load_model
from keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.callbacks import CSVLogger, EarlyStopping
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from mpl_toolkits.axes_grid1 import host_subplot
#from keras_self_attention import SeqSelfAttention
from numpy.lib.stride_tricks import sliding_window_view
#
# tf.logging.set_verbosity(tf.logging.ERROR)
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

### Loading Data

In [2]:
fdat     = '../DataProcessed/15/m-input-output.pkl'
file      = open(fdat, 'rb')
path      = pickle.load(file)
fdat     = pickle.load(file)
lahead    = pickle.load(file)
lpar      =pickle.load(file)
stock_list= pickle.load(file)
tot_res   = pickle.load(file)
df_dict   = tot_res['INP']
win, deep, n_ftrs,tr_tst = lpar

In [3]:
tot_res.keys()

dict_keys(['INP', 'INP_MSERIAL'])

In [4]:
df_dict['TSLA'].index

DatetimeIndex(['2015-01-02', '2015-01-05', '2015-01-06', '2015-01-07',
               '2015-01-08', '2015-01-09', '2015-01-12', '2015-01-13',
               '2015-01-14', '2015-01-15',
               ...
               '2023-12-15', '2023-12-18', '2023-12-19', '2023-12-20',
               '2023-12-21', '2023-12-22', '2023-12-26', '2023-12-27',
               '2023-12-28', '2023-12-29'],
              dtype='datetime64[ns]', name='date', length=2246, freq=None)

In [5]:
mpl.rcParams['figure.figsize'] = (18, 10)
mpl.rcParams['axes.grid'] = False
sns.set_style("whitegrid")

### Transformer

In [32]:
import torch
from torch import nn
from tqdm import tqdm

from EncoderTransformer import TransformerEncoder as Encoder
from DecoderTransformer import TransformerDecoder as Decoder


class Transformer(nn.Module):
    def __init__(
            self,
            num_encoder_layers: int,
            num_decoder_layers: int,
            dim_model: int,
            num_attention_heads: int,
            units_hidden_layer: int,
            dropout: float,
            n_ftrs: int,
            activation: nn.Module = nn.ReLU(),
            mask: bool = True):

        super().__init__()
        self.encoder = Encoder (
            num_layers=num_encoder_layers,
            d_model=dim_model,
            num_heads=num_attention_heads,
            units_hidden_layer=units_hidden_layer,
            dropout=dropout,
            activation=activation,
            mask=mask)

        self.decoder = Decoder (
            num_layers=num_decoder_layers,
            d_model=dim_model,
            num_heads=num_attention_heads,
            units_hidden_layer=units_hidden_layer,
            dropout=dropout,
            activation=activation,
            mask=mask)
        
        self.output_layer = nn.Linear(dim_model, n_ftrs) # Linear layer for predicting the single feature

    def forward(self, source, target):
        outputs = []

        for src, tgt_seq in tqdm(zip(source, target), total=len(source)):
            src = torch.unsqueeze(src, dim=0)
            encoder_output = self.encoder(src)

            batch_output = []
            for tgt_step in tgt_seq:
                tgt_step = torch.unsqueeze(tgt_step, dim=0)
                decoder_output = self.decoder(src=tgt_step, memory=encoder_output)
                batch_output.append(decoder_output)

            # Concatenate along the time dimension and apply the linear layer
            output = torch.cat(batch_output, dim=0)
            output = self.output_layer(output)
            outputs.append(output)

        return torch.cat(outputs, dim=0)

In [55]:
num_encoder_layers = 3
num_decoder_layers = 3
d_model = 256
num_heads = 8
units_hidden_layer = 512
dropout_rate = 0.1

epoch = 200
bsize = 150


In [51]:
import math
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models


def transformer_fun(trainX, trainY, testX, testY, Y, vdd, epoch, bsize, nhead, num_encoder_layers, num_decoder_layers, win, n_ftrs, shft, stock, seed):
    nptrX = trainX #.reshape(trainX.shape[0], trainX.shape[1], n_ftrs)
    nptrY = trainY
    nit = 0
    lloss = np.nan
    
    while math.isnan(lloss) and nit < 5:
        tf.random.set_seed(seed)
        # create a Transformer model
        transformer_model = Transformer(
        num_encoder_layers=num_encoder_layers,
        num_decoder_layers=num_decoder_layers,
        dim_model=d_model,
        num_attention_heads=num_heads,
        units_hidden_layer=units_hidden_layer,
        dropout=dropout_rate,
        n_ftrs=n_ftrs
        ) # Instantiate the Transformer model

        model = models.Sequential()
        model.add(layers.TimeDistributed(transformer_model, input_shape=(nptrX.shape[0], nptrX.shape[1])))
        model.add(layers.GlobalAveragePooling1D())
        model.add(layers.Dense(n_ftrs))  # Output of a single value
        model.compile(loss='mean_squared_error', optimizer='adam')

        # Training
        hist = model.fit(nptrX, nptrY, epochs=epoch, batch_size=bsize, verbose=0)
        lloss = hist.history['loss'][-1]
        nit = nit + 1

    # Predict
    nptstX = testX.to_numpy().reshape(testX.shape[0], testX.shape[1], n_ftrs)
    nptstY = testY.to_numpy()
    res1 = eval(nptstX, nptstY, model, vdd, Y, shft)
    
    df_result = {
        'MSEP': res1.get("msep"),
        'MSEY': res1.get("msey"),
        'Stock': stock,
        'DY': res1.get("Ys"),
        'ALG': 'Transformer',
        'seed': seed,
        'epochs': epoch,
        'num_head': nhead,
        'win': win,
        'ndims': 1,
        'lossh': lloss,
        'nit': nit,
        'model': model
    }
    return df_result


In [52]:
tot_res['INP_MSERIAL']["TSLA"][7]['trX'].shape

(1779, 15, 10)

### Modeling the Transformer

In [54]:
res  = {}
tmod =  "transformer"
res['MODEL'] = tmod
fmdls= '/{}/Models/'.format(win)+tmod+'/'
if not os.path.exists(fmdls):
    os.makedirs(fmdls)
#
for stock in stock_list:
    res[stock] = {}
    for ahead in lahead:
        # print('Training ' + stock)
        trainX = tot_res['INP_MSERIAL'][stock][ahead]['trX']
        trainY = tot_res['INP_MSERIAL'][stock][ahead]['trY']
        testX  = tot_res['INP_MSERIAL'][stock][ahead]['tsX']    
        testY  = tot_res['INP_MSERIAL'][stock][ahead]['tsY']
        Y      = tot_res['INP_MSERIAL'][stock][ahead]['y']
        vdd    = tot_res['INP_MSERIAL'][stock][ahead]['vdd']
        tmpr = []
        
        for irp in range(10):
            seed = random.randint(0,1000)
            transf_start= time.time()
            mdl_name  = '{}-{}-{:03}-{:02}.hd5'.format(tmod,stock,ahead,irp)
            if tmod == "transformer":
                sol   = transformer_fun(trainX,trainY,testX,testY,Y,vdd,epoch,bsize,num_heads,num_encoder_layers,num_decoder_layers,win,n_ftrs,ahead,stock,seed)
            tranf_end  = time.time()
            ttrain    = tranf_end - transf_start
            sol['ttrain'] = ttrain
            sol['epoch']  = epoch
            sol['bsize']  = bsize
            sol['num_heads']    = num_heads
            sol['model'].save(fmdls+mdl_name)
            sol['model']  = fmdls+mdl_name
            # print('   Effort spent: ' + str(ttrain) +' s.')
            # sys.stdout.flush()
            tmpr.append(sol)
        res[stock][ahead] = pd.DataFrame(tmpr)
#
tot_res['OUT_MODEL'] = res

ValueError: Please initialize `TimeDistributed` layer with a `tf.keras.layers.Layer` instance. Received: Transformer(
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-2): 3 x TransformerEncoderLayer(
        (attention): Residual(
          (sublayer): MultiHeadAttentionLayer(
            (attention_heads): ModuleList(
              (0-7): 8 x AttentionHead(
                (query): Linear(in_features=256, out_features=32, bias=True)
                (key): Linear(in_features=256, out_features=32, bias=True)
                (value): Linear(in_features=256, out_features=32, bias=True)
              )
            )
            (linear_layer): Linear(in_features=256, out_features=256, bias=True)
          )
          (norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (feed_forward): Residual(
          (sublayer): Sequential(
            (0): Linear(in_features=256, out_features=512, bias=Tru